In [160]:
import pandas as pd
import numpy
from tqdm import tqdm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


new = pd.DataFrame()
df_predictions = pd.DataFrame

df_accuracy = pd.read_csv('top_and_bottom_anime.csv')
df_accuracy = df_accuracy.drop(columns='Id')
df_accuracy = df_accuracy.drop(columns='Rating')
df_accuracy = df_accuracy.drop(columns='Demographic')


le = preprocessing.LabelEncoder()
for i in range(df_accuracy.shape[1]):
    temp = df_accuracy.iloc[:,i].values.tolist()
    temp = le.fit_transform(temp)
    new[df_accuracy.columns[i]] = temp

df_true = pd.read_csv('anime_matrix_true.csv')
df_predictions = df_true[['Id']].copy()
df_true = df_true.drop(columns='Id')




In [140]:
columns = df_true.columns
predictions = []
true = []
# x = new
# y = df_true[columns[1]]
# x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=42)

# #create cross training split for building the models
# X_cross_train, X_cross_test, y_cross_train, y_cross_test = train_test_split(x_train, y_train, test_size=0.2, random_state=420)
avg = []
for i in tqdm(range(len(columns))):
    x = new
    y = df_true[columns[i]]
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=42)
    # y_train = df_true[columns[i]][:len(x_train)]
    #create cross training split for building the models
    X_cross_train, X_cross_test, y_cross_train, y_cross_test = train_test_split(x_train, y_train, test_size=0.2, random_state=420)

    adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    adaboost.fit(X_cross_train,y_cross_train)
    avg.append(adaboost.score(X_cross_test,y_cross_test))
    predictions.append(list(y_cross_test))

# result

100%|██████████| 9029/9029 [12:26<00:00, 12.09it/s]


In [144]:
print(sum(avg) / len(avg))

0.8261755815530566


In [142]:
import warnings

cut_off = (len(df_predictions)) - (len(y_cross_test))
df_predictions.drop(df_predictions.tail(cut_off).index,inplace=True)

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
# df_predictions = pd.DataFrame()
for i in tqdm(range(len(columns))):
    df_predictions[columns[i]] = predictions[i]

100%|██████████| 9029/9029 [00:04<00:00, 2041.83it/s]


In [130]:
import pickle

# open a file, where you ant to store the data
file = open('adaboost.model', 'wb')

# dump information to that file
pickle.dump(adaboost, file)

# close the file
file.close()

In [143]:
outputdf = df_predictions.copy()
outputdf.to_csv(f'predictions_matrix_adaboost.csv', index=False)

In [178]:
#creates the true values to check the accuracy in the cell below
#uses the same splits with the same seeds to grab which anime is in x_cross_test
x = new
y = df_predictions[['Id']]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=42)
X_cross_train, X_cross_test, y_cross_train, y_cross_test = train_test_split(x_train, y_train, test_size=0.2, random_state=420)

df_true = pd.read_csv('anime_matrix_true.csv')
temp = df_true.values
true = []
for each in list(y_cross_test['Id']):
    for j in range(len(temp)):
        if each in list(temp[j]):
            true.append(list(temp[j]))

true = numpy.array([numpy.array(xi) for xi in true])

In [180]:
#caluclate accuracy
value = 0
prediction_matrix = pd.read_csv('predictions_matrix_adaboost.csv')
pred = prediction_matrix.values

num_row = len(pred)
num_col = len(pred[0])
for i in tqdm(range(num_row)):
    for j in range(1,num_col):          #first item is the anime id, so we skip it
        if 0 == true[i,j]:
            value += 1

print(value)
print(value / ((num_row) * (num_col)))
# print('zeros = ',zeros)
# print('ones = ',ones)
# print('true ones = ',true_ones)
print((num_row) * (num_col))

100%|██████████| 89/89 [00:00<00:00, 740.99it/s]

659053
0.8201450756053217
803581


In [170]:
print(pred[1,0])

26453
